In [27]:
import urllib
import re
import requests
from urllib.request import Request, urlopen
import numpy as np
import json
import datetime
from dateutil.parser import parse
import pprint
pp = pprint.PrettyPrinter(indent=4)
popUpList = []
information = ["vendor-name", 'type', 'price', 'attribute', 'about', 'menu', 'schedule', 'chef-info', 'stars-count', 'review-count', 'cuisine', 'cuisine-map', 'schedule']
eventVars = ['location', 'address', 'city', 'state', 'latitude', 'longitude', 'times']
timeVars = ['month', 'day', 'startTime', 'endTime']

In [28]:
def price_band(int_):
    if int_ < 25.00:
        return 1
    elif int_ < 35.00:
        return 2
    elif int_ < 60.00:
        return 3
    else:
        return 4

price_band(45)

3

In [29]:
url = 'https://eatfeastly.com/api/v1/meals/upcoming/?non_private=true&market=san-francisco'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
r = urlopen(req).read()
j = json.loads(r)['results']
for popUp in j:
    infoObj = dict.fromkeys(information)
    timeObj = dict.fromkeys(timeVars)
    timeObj['month'] = []
    timeObj['day'] = []
    timeObj['startTime'] = []
    timeObj['endTime'] = []
    eventObj = dict.fromkeys(eventVars)
    eventObj['times'] = timeObj
    infoObj['schedule'] = []
    
    popUpID = str(popUp['pk'])
    popUpUrl = 'https://eatfeastly.com/api/v1/meals/' + popUpID + '/?view=all'
    req = Request(popUpUrl, headers={'User-Agent': 'Mozilla/5.0'})
    r = urlopen(req).read()
    j = json.loads(r)
    
    infoObj['about'] = j['description']
    infoObj['vendor-name'] = j['title']
    infoObj['chef-info'] = j['cook']['about']
    cuisine = []
    for type in j['cuisine_type']:
        cuisine.append(type)
    infoObj['cuisine'] = cuisine
    menuStr = ''
    for dish in j['dishes']:
        menuStr = menuStr + dish['about'] + ' ' + dish['course'] + ' ' + dish['dish_title'] + ' '
    infoObj['menu'] = menuStr
    infoObj['price'] = price_band(float(j['price'][0]['price_feastly']))
    infoObj['review-count'] = j['cook']['review_count']
    eventObj['address'] = j['neighborhood']
    eventObj['city'] = j['location']['market']
    eventObj['latitude'] = j['location']['lat']
    eventObj['longitude'] = j['location']['lng']
    eventObj['state'] = j['location']['state']
    eventObj['times']['day'].append(parse(j['event_data']['start_time']).day)
    eventObj['times']['month'].append(parse(j['event_data']['start_time']).month)
    eventObj['times']['startTime'].append(parse(j['event_data']['start_time']).hour)
    duration = datetime.timedelta(minutes = j['event_data']['duration'])
    endTime = parse(j['event_data']['start_time']) + duration
    eventObj['times']['endTime'].append(endTime.hour)
    infoObj['schedule'].append(eventObj)
    infoObj['stars-count'] = j['cook']['review_avg']
    infoObj['type'] = 'popup'
    infoObj['attribute'] = {}
    infoObj['cuisine']
    
    popUpList.append(infoObj)

popUpList

[{'about': "What you're about to experience is as if you were in Iran trying a homemade meal.\n\nIn this menu, I'd like to introduce you to a unique culinary gem from my beloved Iran.\n\nNoosh e joon!",
  'attribute': None,
  'chef-info': '"People who love to eat are always the best people" ~ Julia Child\r\n\r\nSalam! My love for cooking started at a young age in my motherland of Iran, when I used to watch my mom cook for our family of six and endless more guests! She taught me to cook with passion and educate myself about the healing power of food and herbs. To this day, healthy eating habits play an important role in my cooking and personal lifestyle. \r\n\r\nWe had a beautiful tradition at home of making our own delicate pastries for the Persian New Year, Nowruz, marking the first day of Spring. Family and friends simply loved it, and we loved spreading the joy! I’ve had the chance to explore and share my cooking and baking skills since then. When I was 16, I volunteered to help rai

In [30]:
popUpJSON = json.dumps(popUpList)
with open('popup-data.txt', 'w') as outfile:
    json.dump(popUpJSON, outfile)

In [31]:

pp.pprint(popUpJSON)

('[{"vendor-name": "Seti\'s Table: The Saffron Story, 5 Course", "type": '
 '"popup", "price": 4, "attribute": null, "about": "What you\'re about to '
 'experience is as if you were in Iran trying a homemade meal.\\n\\nIn this '
 "menu, I'd like to introduce you to a unique culinary gem from my beloved "
 'Iran.\\n\\nNoosh e joon!", "menu": "Welcome refreshment, Seti\\u2019s '
 'signature Orange Blossom Chia Seed Drink Beverage Orange Blossom Drink Fresh '
 'herbs, Mix greens, Bulgur, Pistachio, Green Raisin with Orange Blossom Lime '
 'and Olive oil Dressing Appetizer Pistachio Salad Specialty from Shiraz, this '
 'dish is a beautiful saffron and yogurt infused Entree. Rice, saffron and  '
 "yogurt. I'll add a layer of roasted eggplant and get ready to try the crispy "
 'Tahdig! Entree Tahcheen Marinated chicken with yogurt is one of the most '
 'common food in Iran. Either Bbq or roasting in the oven.Yogurt will make the '
 'chicken so tender and juicy. \\n \\nThis dish will be serve

 "romesco, charred scallion salsa Small Plate Trumpet 'scallops' Shaved and "
 'massaged mustard greens, crispy sunchokes, tahini-lemon drizzle, smoked '
 'Maldon salt Small Plate Mustard greens Handmade parsnip and white potato '
 'gnocchi, garlicky cauliflower Alfredo sauce, fresh spring peas, fried '
 'onions, golden broiled panko crust Entree Parsnip Gnocchi Rich chocolate '
 'cake with a gooey chocolate center and spiced cherry jam Dessert Chocolate '
 'Lava Cakes ", "schedule": [{"location": null, "address": "Nob Hill", "city": '
 '"san-francisco", "state": "California", "latitude": 37.77493, "longitude": '
 '-122.41942, "times": {"month": [4], "day": [24], "startTime": [19], '
 '"endTime": [21]}}], "chef-info": "Hello! I am a plant based chef in the East '
 'Bay looking to share the bounty of flavorful clean eating. I have cooked at '
 'both traditional and vegan restaurants and as a caterer around the bay for '
 'the last four years. My focus is healing through nutrition and th

In [32]:
pp.pprint (json.loads(popUpJSON)[1])

{   'about': 'Born and raised in the bay area however, I spent a good amount '
             'of my childhood in Hawaii. My passion for cooking started at a '
             'young age. I was introduced to cooking by my grandparents and '
             "parents. I've been told by my family that I share my "
             "grandfather's passion for cooking. \n"
             '\n'
             'This menu is inspired by some of my favorite Filipino dishes '
             'that my grandparents introduced me to.',
    'attribute': None,
    'chef-info': 'Born and raised in the bay area however, I spent a good '
                 'amount of my childhood in Hawaii. My passion for cooking '
                 'started at a young age. I was introduced to cooking by my '
                 "grandparents and parents. I've been told by my family that I "
                 "share my grandfather's passion for cooking. My grandfather "
                 'would prepare meals for large groups of people using '
     